# Maximum Flows

We wish to transport material from node 0 (the source) to node 4 (the sink). The numbers next to the arcs are their capacities — the capacity of an arc is the maximum amount that can be transported across it in a fixed period of time.<br>

The max flow problem is to find a flow for which the sum of the flow amounts for the entire network is as large as possible.

In [1]:
"""From Taha 'Introduction to Operations Research', example 6.4-2."""

from __future__ import print_function
from ortools.graph import pywrapgraph

def main():
  """MaxFlow simple interface example."""

  # Define three parallel arrays: start_nodes, end_nodes, and the capacities
  # between each pair. For instance, the arc from node 0 to node 1 has a
  # capacity of 20.

  start_nodes = [0, 0, 0, 1, 1, 2, 2, 3, 3]
  end_nodes = [1, 2, 3, 2, 4, 3, 4, 2, 4]
  capacities = [20, 30, 10, 40, 30, 10, 20, 5, 20]

  # Instantiate a SimpleMaxFlow solver.
  max_flow = pywrapgraph.SimpleMaxFlow()
  # Add each arc.
  for i in range(0, len(start_nodes)):
    max_flow.AddArcWithCapacity(start_nodes[i], end_nodes[i], capacities[i])

  # Find the maximum flow between node 0 and node 4.
  if max_flow.Solve(0, 4) == max_flow.OPTIMAL:
    print('Max flow:', max_flow.OptimalFlow())
    print('')
    print('  Arc    Flow / Capacity')
    for i in range(max_flow.NumArcs()):
      print('%1s -> %1s   %3s  / %3s' % (
          max_flow.Tail(i),
          max_flow.Head(i),
          max_flow.Flow(i),
          max_flow.Capacity(i)))
    print('Source side min-cut:', max_flow.GetSourceSideMinCut())
    print('Sink side min-cut:', max_flow.GetSinkSideMinCut())
  else:
    print('There was an issue with the max flow input.')

main()

Max flow: 60

  Arc    Flow / Capacity
0 -> 1    20  /  20
0 -> 2    30  /  30
0 -> 3    10  /  10
1 -> 2     0  /  40
1 -> 4    20  /  30
2 -> 3    10  /  10
2 -> 4    20  /  20
3 -> 2     0  /   5
3 -> 4    20  /  20
Source side min-cut: [0]
Sink side min-cut: [4, 1]


# Assignment as a Minimum Cost Flow Problem

## Linear assignment example

This section show how to solve the example, described in the section Linear Assignment Solver (worker as a source and task as a sink), as a min cost flow problem.

In [3]:
from __future__ import print_function
from ortools.graph import pywrapgraph
import time

def main():
  min_cost_flow = pywrapgraph.SimpleMinCostFlow()
  
  # Define the directed graph for the flow.
  team_A = [1, 3, 5]
  team_B = [2, 4, 6]

  start_nodes = ([0, 0]  + [11, 11, 11] + [12, 12, 12] +
                 [1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 6, 6, 6, 6] +
                 [7, 8, 9, 10])
  end_nodes =   ([11, 12] + team_A + team_B +
                 [7, 8, 9, 10, 7, 8, 9, 10, 7, 8, 9, 10, 7, 8, 9, 10, 7, 8, 9, 10, 7, 8, 9, 10] +
                 [13, 13, 13, 13])
  capacities =  ([2, 2] + [1, 1, 1] + [1, 1, 1] +
                 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] +
                 [1, 1, 1, 1])
  costs      =  ([0, 0] + [0, 0, 0] + [0, 0, 0] +
                 [90, 76, 75, 70, 35, 85, 55, 65, 125, 95, 90, 105, 45, 110, 95, 115, 60, 105,
                  80, 75, 45, 65, 110, 95] + [0, 0, 0, 0])

  # Define an array of supplies at each node.

  supplies = [4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -4]
  source = 0
  sink = 13

  # Add each arc.
  for i in range(0, len(start_nodes)):
    min_cost_flow.AddArcWithCapacityAndUnitCost(start_nodes[i], end_nodes[i],
                                                capacities[i], costs[i])

  # Add node supplies.

  for i in range(0, len(supplies)):
    min_cost_flow.SetNodeSupply(i, supplies[i])
  # Find the minimum cost flow between node 0 and node 10.
  if min_cost_flow.Solve() == min_cost_flow.OPTIMAL:
    min_cost_flow.Solve()
    print('Total cost = ', min_cost_flow.OptimalCost())
    print()

    for arc in range(min_cost_flow.NumArcs()):

      # Can ignore arcs leading out of source or intermediate nodes, or into sink.
      if (min_cost_flow.Tail(arc)!=0 and min_cost_flow.Tail(arc)!=11 and min_cost_flow.Tail(arc)!=12
          and min_cost_flow.Head(arc)!=13):

        # Arcs in the solution will have a flow value of 1. There start and end nodes
        # give an assignment of worker to task.

        if min_cost_flow.Flow(arc) > 0:
          print('Worker %d assigned to task %d.  Cost = %d' % (
                min_cost_flow.Tail(arc),
                min_cost_flow.Head(arc),
                min_cost_flow.UnitCost(arc)))
  else:
    print('There was an issue with the min cost flow input.')

main()

Total cost =  250

Worker 1 assigned to task 9.  Cost = 75
Worker 2 assigned to task 7.  Cost = 35
Worker 5 assigned to task 10.  Cost = 75
Worker 6 assigned to task 8.  Cost = 65
